In [1]:
!pip install Counter
!pip install torch
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for Counter: filename=Counter-1.0.0-py3-none-any.whl size=5411 sha256=ef4233741745fb3bad1d4843e7b0edac8cece3b00834575f01ec3d959674c365
  Stored in directory: /root/.cache/pip/wheels/05/78/ee/7f4336de4d2631224a4b303be03e48eeb01db89bb5bd790f5d
Successfully built Counter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
TRAIN_TEXT_FILE_PATH = '/content/train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'н', 'и', 'т', 'р', 'с', 'в', 'м', 'л', 'п', 'к', 'я', 'б', 'д', 'ы', 'у', '.', 'з', '\n', 'й', '"', 'ц', 'г', '\t', 'ч', ',', 'ь', 'ф', 'х', 'ю', 'В', 'щ', 'ж', 'ш', 'О', '«', '»', 'ъ', 'З', 'Р', ':', 'Н', '1', 'С', 'П', '-', '2', 'ё', 'К', 'M', 'И', 'А', 'Е', 'i', 'o', 'U', 'L', 'Д', 't', 'Ц', 'э', 'e', 'Т', 'a', 'B', 's', 'Б', 'У', 'r', 'P', '(', ')', 'S', 'l', 'c', 'd', 'p', 'I', 'K', 'n', ';', 'T', '3', 'u', 'm', 'A', 'Г', 'D', 'g', 'h', 'Ф', 'Ж', 'f', 'Л', 'R', '–', 'k', 'V', 'w', 'y', 'b', 'E', 'М', '—', 'Ч', 'F', '+', 'Э', 'C', '4', '5', '6', 'O', 'N', 'G', 'x', 'W']


In [4]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [5]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [6]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.700881552696228
 работе соремы сторования данная работе соремения прования прования работе соответстветветветствует сования квалия работа полно».	"Выполной соответстветветствует требования прованнана работе соремения 
Loss: 1.3529711639881135
 проектам, а предъявляемым к выпускной 
 квалификационной работа полностью соответствует требованиям, предъявляемым к выпускной работа полностью соответствует требованиям, предъявляемым к выпускной 
 к
Loss: 0.9495746386051178
 программной системы компоненты посвящена разработке системы посвящена разработке автоматизированного бакалавра."	Оценка «отлично».	Замечаний по работе нет.	
 Разработке представленная работа полностью
Loss: 0.7583040237426758
 использование программного проработана масштабируемость системы компоненты программной системы управления программной системы для области в разработка проектирование и разработке автоматизации UML, на
Loss: 0.627738550901413
 специалиста."	Оценка «хорошо».	"Замечания по работе:
 1. Не достаточ

KeyboardInterrupt: ignored